In [0]:
trigger_location = dbutils.widgets.get("trigger_location")
prompt = dbutils.widgets.get("prompt")
frame_stride = int(dbutils.widgets.get("frame_stride"))
truncate = dbutils.widgets.get("truncate")
threshold = float(dbutils.widgets.get("threshold"))

# trigger_location = '/Volumes/pubsec_video_processing/cv/auto_segment/images/maren_jack.MOV'
# prompt = 'girl in pink sweatshrit'
# frame_stride = 30
# truncate = True
# threshold = 0.2

if truncate==0 or truncate=='false' or truncate=='False':
  truncate = False
else:
  truncate=True

print(trigger_location)
print(prompt)
print(frame_stride)
print(truncate)
print(type(truncate))
print(threshold)
print(type(threshold))

In [0]:
# dbutils.notebook.exit("Notebook execution stopped by user request.")

In [0]:
import mlflow
import imageio
import numpy as np
import pandas as pd
import cv2
import base64
import timeit
from io import BytesIO
from PIL import Image
import os

In [0]:
from datetime import datetime

if trigger_location.endswith('/') and (trigger_location[-4]!='.' or trigger_location[-5]!='.'):
  # Your volume directory path
  directory_path = trigger_location

  # List all files in the directory
  files = dbutils.fs.ls(directory_path)

  # Filter out directories, keep only files
  files = [f for f in files if not f.isDir()]

  # Sort by modification time (most recent first)
  files_sorted = sorted(files, key=lambda x: x.modificationTime, reverse=True)

  # Get the most recent file
  if files_sorted:
      most_recent_file = files_sorted[0]
      most_recent_path = most_recent_file.path.replace('dbfs:','')
      most_recent_name = most_recent_file.name
      
      print(f"Most recent file: {most_recent_name}")
      print(f"Full path: {most_recent_path}")
      print(f"Modified: {datetime.fromtimestamp(most_recent_file.modificationTime/1000)}")
  else:
      print("No files found in directory")
else:
  most_recent_file = dbutils.fs.ls(trigger_location)[0]
  most_recent_path = most_recent_file.path.replace('dbfs:','')
  most_recent_name = most_recent_file.name

  print(f"Most recent file: {most_recent_name}")
  print(f"Full path: {most_recent_path}")
  print(f"Modified: {datetime.fromtimestamp(most_recent_file.modificationTime/1000)}")

In [0]:
# most_recent_file = trigger_location
# most_recent_name = most_recent_file.split("/")[-1]
# most_recent_path = most_recent_file.replace('dbfs:','')

In [0]:
from huggingface_hub import login
import os

hf_pat = dbutils.secrets.get("justinm-buildathon-secrets", "hf_pat")
os.environ["HF_TOKEN"] = hf_pat
login(token=hf_pat)

In [0]:
import mlflow

model = mlflow.pyfunc.load_model("models:/pubsec_video_processing.cv.transformers-sam3-video@job")

In [0]:
def write_results(FILE_URL, results):
  import os

  OUTPUT_FILE_URL = FILE_URL.replace("inputs", "outputs")
  output_dir = os.path.dirname(OUTPUT_FILE_URL)
  os.makedirs(output_dir, exist_ok=True)
  FPS = 24

  def decode_mask(encoded_mask: str) -> np.ndarray:
      """Decode base64 mask back to numpy array"""
      buf = BytesIO(base64.b64decode(encoded_mask))
      return np.load(buf)

  def add_timestamp(frame: np.ndarray, timestamp_sec: float) -> np.ndarray:
      """Add timestamp overlay to frame in the top-right corner"""
      # Convert seconds to HH:MM:SS.ms format
      hours = int(timestamp_sec // 3600)
      minutes = int((timestamp_sec % 3600) // 60)
      seconds = int(timestamp_sec % 60)
      milliseconds = int((timestamp_sec % 1) * 1000)
      
      timestamp_text = f"{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}"
      
      # Create a copy to avoid modifying original
      frame_with_timestamp = frame.copy()
      
      # Get frame dimensions
      height, width = frame.shape[:2]
      
      # Set up text properties
      font = cv2.FONT_HERSHEY_SIMPLEX
      font_scale = 0.7
      font_thickness = 2
      text_color = (255, 255, 255)  # White text
      bg_color = (0, 0, 0)  # Black background
      padding = 10
      
      # Get text size
      (text_width, text_height), baseline = cv2.getTextSize(
          timestamp_text, font, font_scale, font_thickness
      )
      
      # Position in top-right corner
      text_x = width - text_width - padding
      text_y = padding + text_height
      
      # Draw semi-transparent background rectangle
      bg_x1 = text_x - 5
      bg_y1 = text_y - text_height - 5
      bg_x2 = text_x + text_width + 5
      bg_y2 = text_y + baseline + 5
      
      # Create overlay for semi-transparency
      overlay = frame_with_timestamp.copy()
      cv2.rectangle(overlay, (bg_x1, bg_y1), (bg_x2, bg_y2), bg_color, -1)
      cv2.addWeighted(overlay, 0.6, frame_with_timestamp, 0.4, 0, frame_with_timestamp)
      
      # Draw text
      cv2.putText(
          frame_with_timestamp,
          timestamp_text,
          (text_x, text_y),
          font,
          font_scale,
          text_color,
          font_thickness,
          cv2.LINE_AA
      )
      
      return frame_with_timestamp

  # Open original video to get frames
  print("Processing frames and applying masks...")
  cap = cv2.VideoCapture(FILE_URL)
  fps = cap.get(cv2.CAP_PROP_FPS) or FPS

  # Create a mapping of frame_idx to results
  print(f"Found {len(results)} frames in results")
  result_map = {r["frame_idx"]: r for r in results}

  frame_idx = 0
  saved_images = []
  segmented_images = []

  while cap.isOpened():
      ret, frame = cap.read()
      if not ret:
          break
      
      # Only process frames with segmentation results
      if frame_idx in result_map:
          # Convert BGR to RGB
          rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          
          res = result_map[frame_idx]
          
          # if res["masks"]:
          #     # Get the first (highest score) mask
          #     mask = decode_mask(res["masks"][0])
              
          #     # Overlay with transparency
          #     overlay = rgb_frame.copy()
          #     overlay[mask > 0.5] = [0, 255, 0]  # Green overlay
          #     masked_frame = cv2.addWeighted(rgb_frame, 0.7, overlay, 0.3, 0)

          if res["masks"]:
              # Get the first (highest score) mask
              masks = [decode_mask(x) for x in res["masks"]]
              
              # Overlay with transparency
              overlay = rgb_frame.copy()
              for mask in masks:
                  overlay[mask > 0.5] = [0, 255, 0]  # Green overlay
              masked_frame = cv2.addWeighted(rgb_frame, 0.7, overlay, 0.3, 0)
              
          else:
              masked_frame = rgb_frame
          
          # Calculate timestamp for this frame
          timestamp_sec = frame_idx / fps
          
          # Add timestamp overlay (convert back to BGR for cv2 operations, then back to RGB)
          masked_frame_bgr = cv2.cvtColor(masked_frame, cv2.COLOR_RGB2BGR)
          masked_frame_with_timestamp = add_timestamp(masked_frame_bgr, timestamp_sec)
          masked_frame = cv2.cvtColor(masked_frame_with_timestamp, cv2.COLOR_BGR2RGB)
          
          # Save frame
          if not truncate:
              saved_images.append(Image.fromarray(rgb_frame))
              segmented_images.append(Image.fromarray(masked_frame))
          elif res["masks"]:
              saved_images.append(Image.fromarray(rgb_frame))
              segmented_images.append(Image.fromarray(masked_frame))
      
      frame_idx += 1

  cap.release()
  print(f"Saved {len(saved_images)} frames to memory")
  print(f"Saved {len(segmented_images)} segmented frames to memory")

  # 3. Create full segmented video
  import imageio
  import os
  import shutil
  import tempfile

  print("Writing video to temporary file...")
  with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as tmp_file:
      temp_video_path = tmp_file.name

  imageio.mimsave(
      temp_video_path,
      segmented_images,
      fps=(fps/frame_stride) * 5,  # Use original FPS instead of hardcoded 24
      codec='libx264',
      pixelformat='yuv420p'
  )

  temp_size = os.path.getsize(temp_video_path)
  print(f"Temporary video created: {temp_size:,} bytes ({temp_size/1024/1024:.2f} MB)")

  # Copy to Volumes
  print(f"Copying to Volumes: {OUTPUT_FILE_URL}")
  shutil.copy2(temp_video_path, OUTPUT_FILE_URL)

  final_size = os.path.getsize(OUTPUT_FILE_URL)
  print(f"✓ Video successfully saved to: {OUTPUT_FILE_URL}")
  print(f"  Final size: {final_size:,} bytes ({final_size/1024/1024:.2f} MB)")

  # Clean up temporary file
  if os.path.exists(temp_video_path):
      os.remove(temp_video_path)
      print("Cleaned up temporary file")

  return saved_images

In [0]:
def process_file(triggered_file, prompt):
    print(f"Triggered by file: {triggered_file}")
    # Your processing logic here
    model_input = pd.DataFrame([{
        "video_path": triggered_file,
        "prompt": prompt,
        "frame_stride": frame_stride,  # Process every nth frame
        "batch_size": 16,
        "threshold": threshold,
        "mask_threshold": 0.5
    }])
    results = model.predict(model_input)
    return results

In [0]:
print("Files processed in this run:", most_recent_path)

print("Segmenting video...")
starting_time = timeit.default_timer()
results = process_file(most_recent_path, prompt)
print(f"Inference time: {round((timeit.default_timer() - starting_time))} secs")

In [0]:
written_images = write_results(most_recent_path, results)

In [0]:
captioning_model = mlflow.pyfunc.load_model("models:/pubsec_video_processing.cv.transformers-blip@job")

In [0]:
from PIL import Image
from io import BytesIO
import base64

def pil_to_base64_str(img: Image.Image, format: str = "PNG") -> str:
    """
    Convert a PIL image to a Base64-encoded string.
    
    Args:
        img: PIL.Image.Image
        format: image format, e.g., 'PNG' or 'JPEG'
        
    Returns:
        str: Base64 string that can be safely passed in JSON
    """
    buf = BytesIO()
    img.save(buf, format=format)
    buf.seek(0)
    b64_str = base64.b64encode(buf.read()).decode("utf-8")
    return b64_str

In [0]:
from io import BytesIO

written_images_str = [pil_to_base64_str(x) for x in written_images]

In [0]:
model_input = {
    "image_path": written_images_str
}

captions = captioning_model.predict(model_input)

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

# Initialize the client
w = WorkspaceClient()

# Define your endpoint name
endpoint_name = "databricks-gpt-5-2"

# # Create a chat message
# messages = [
#     ChatMessage(
#         role=ChatMessageRole.USER,
#         content=f"The following is a list of descriptions of images. The images represent frames from a video, in order. Can you identify anything out of the ordinary or anomalous in the video? When I say anomlous, I mean actions taking place that are against the law or violent in nature. If you find something that shouldn't be happening, describe why it is anomolous. If you don't find anything abnormal, just respond with 'Nothing anomolous found.' \n\n {captions}"
#     )
# ]
# Create a chat message
captions = list(set(captions))
prompt = f"""Summarize the following sections of a video with a focus on what is functionally happening over time. Be descriptive. The resulting summary should be a functional narrative. Return this as Markdown text - and only markdown. \n\n {captions}"""

# messages = [
#     ChatMessage(
#         role=ChatMessageRole.USER,
#         content=f"The following is a list of descriptions of images. The images represent frames from a video, in order. There may be obvious scene changes based on the descriptions. Please summarize the contents of each scene in the video in a single sentence each.' \n\n {captions}"
#     )
# ]

messages = [
    ChatMessage(
        role=ChatMessageRole.USER,
        content=prompt
    )
]

# Query the endpoint
response = w.serving_endpoints.query(
    name=endpoint_name,
    messages=messages,
    # max_tokens=500  # optional parameter
)

# Access the response
r = response.choices[0].message.content
# print(r)
text = [x for x in r if x['type'] == 'text'][0]['text']
print(text)

  File <command-7680324554620704>, line 19
    'Summarize the following sections of a video with a focus on '
    ^
IndentationError: unexpected indent


In [0]:
txt_filename = most_recent_path.split('.')[0] + '.txt'
txt_filename = txt_filename.replace('/inputs/', '/descriptions/')
print(txt_filename)

dbutils.fs.put(txt_filename, text, True)